# Extracció d'entitats anomenades

L'objectiu d'aquesta pràctica era fer un reconeixedor d'entitats anomenades amb conditional random fields. També experimentar amb diferents "features_funcionts" i amb diferents tipus de codificacions. Nosaltres vam estructurar l'experimentació de la següent forma:

- Primer vam provar diferents tipus de funcions per decidir quines utilitzar

- En segon lloc, vam provar diferents contexts en els quals aplicar aquestes funcions

- Després vam posar a prova ambdós models a les diferents codificacions per escollir-ne una per cada idioma.

-Un cop els models van estar fets, vam provar-los a la partició de test

A continuació el codi i explicació de cada bloc de la nostra pràctica.

## 1 - Experimentació de "feature functions"

### Import de les dades i preprocessament

En primer lloc, vam importar les dades d'entrenament, validació i test de cada un dels dos idiomes. Aquestes dades venien en la codificació "BIO". Més endavant aquests tags es tractarien per provar altres codificacions.

In [2]:
from typing import List
import nltk
nltk.download('conll2002')
from nltk.corpus import conll2002
import warnings
warnings.filterwarnings('ignore')

[nltk_data] Downloading package conll2002 to
[nltk_data]     C:\Users\maxmg\AppData\Roaming\nltk_data...
[nltk_data]   Package conll2002 is already up-to-date!


In [67]:
esp_train = conll2002.iob_sents('esp.train') # Train
esp_testa = conll2002.iob_sents('esp.testa') # Development
esp_testb = conll2002.iob_sents('esp.testb') # Test

ned_train = conll2002.iob_sents('ned.train') # Train
ned_testa = conll2002.iob_sents('ned.testa') # Development
ned_testb = conll2002.iob_sents('ned.testb') # Test

El preprocessing el vam idear molt centrats en els passos següents. Sabíem que voldríem experimentar amb diferents combinacions "feature functions", i aplicades a diferents contextos dels tokens, i per tant imaginàvem que acabaríem creant un alt nombre de models.

Per tal d'agilitzar-ho i de fer-ho de la manera més eficient possible, vam decidir calcular les "feature functions" de cada token al "preprocessing". Aquest plantejament implicava dos principals avantatges:

- La primera és que en el cas de voler agafar informació del context d'un token, cosa que sabíem que acabaríem fent, no caldria recalcular vàries vegades les funcions d'un token, només caldria accedir a un índex diferent i d'allà prendre les "feature functions" convenients. 

- La segona es que en cas de fer "grid search" també ens estalviaríem molt de temps de computació, ja que no caldria que es calculessin les "feature functions" a cada model, només es calculen un cop, i els diferents models el que fan és agafar-les o no agafar-les.

Així que el primer pas que vam haver de fer és definir quines funcions es calculen per cada token. Les 5 funcions bàsiques les vam mantenir i en vam afegir 5 funcions noves:

1. Prefixos fins a longitud 3

2. El Pos-Tag del token

3. La longitud del token

4. Si el token és una stop-word

Utilitzant llistes externes:

5. Si el token es troba en una llista de noms, organitzacions o localitzacions típiques

Així que l'objectiu final del preprocessing seria transformar les dades importades perquè es converteixin en una tupla del token i uns llista de tres llistes de funcions, en primer lloc les bàsiques, en segon lloc les quatres primeres extres i en tercer lloc, les funcions de les llistes externes. Aquest format ens permetria més endavant activar i desactivar les funcions extres i el context usat de forma senzilla.

Així que un cop establert els objectius del preprocessing i el format desitjat després d'aplicar el mateix, vam crear una sèrie de classes i funcions per aplicar-lo a les dades.

In [65]:
import re
import unicodedata
import nltk
import json
nltk.download('stopwords')


class FuncPreprocessing:
    def __init__(self, language='spanish'):
        self._pattern = re.compile(r"\d")
        if language == 'spanish':
            self.stop_words = set(nltk.corpus.stopwords.words('spanish'))
        elif language == 'dutch':
            self.stop_words = set(nltk.corpus.stopwords.words('dutch'))
        elif language == 'english':
            self.stop_words = set(nltk.corpus.stopwords.words('english'))

        with open('hol_names.txt', 'r', encoding='utf-8') as file:
            hol_names = [line.strip() for line in file.readlines()]
        with open('esp_names.txt', 'r', encoding='utf-8') as file:
            esp_names = [line.strip() for line in file.readlines()]
        self.names = hol_names + esp_names

        with open('countries.json', 'r', encoding='utf-8-sig') as file:
            countries = json.load(file)['countries']
        with open('cities.json', 'r', encoding='utf-8-sig') as file:
            cities = json.load(file)['cities']
        self.locations = [country['name'] for country in countries] + [city['name'] for city in cities]
            

    def __call__(self, corpus: List[List[tuple[str, str]]]) -> List[List[tuple[str, List[List[str]]]]]:
        """
        Extract features from a corpus

        :param corpus: list of list of tuples (word, tag)
        :type corpus: list(list(tuple(str, str)))
        :return: list of features
        :rtype: list(list(list(str)))
        """
        return [self.find_sent_features(sentence) for sentence in corpus]
        
    def find_sent_features(self, sentence):
        """
        Extract features from a sentence

        :param sentence: list of tuples (word, tag)
        :type sentence: list(tuple(str, str))
        :return: list of features
        :rtype: list(list(str))
        """
        new_sentence = []
        for i in range(len(sentence)):
            new_sentence.append(((sentence[i][0][0], self.get_token_features(sentence, i)), sentence[i][1]))
        return new_sentence


    def get_token_features(self, tokens, idx):
        """
        Extract basic features and extra feature about this word including
            - Current word
            - is it capitalized?
            - Does it have punctuation?
            - Does it have a number?
            - Suffixes up to length 3
            - Prefixes up to length 3
            - POS
            - Word length
            - Stop word
            - Names
            - Locations

        :return: a list which contains the features
        :rtype: list(list(str))
        """
        token = tokens[idx][0]

        feature_list = [[], [], []]

        if not token[0]:
            return feature_list

        # Capitalization
        if token[0][0].isupper():
            feature_list[0].append("CAPITALIZATION")

        # Number
        if re.search(self._pattern, token[0]) is not None:
            feature_list[0].append("HAS_NUM")

        # Punctuation
        punc_cat = {"Pc", "Pd", "Ps", "Pe", "Pi", "Pf", "Po"}
        if all(unicodedata.category(x) in punc_cat for x in token[0]):
            feature_list[0].append("PUNCTUATION")

        # Suffix up to length 3
        if len(token[0]) > 1:
            feature_list[0].append("SUF_" + token[0][-1:])
        if len(token[0]) > 2:
            feature_list[0].append("SUF_" + token[0][-2:])
        if len(token[0]) > 3:
            feature_list[0].append("SUF_" + token[0][-3:])

        # Word 

        feature_list[0].append("WORD_" + token[0])

        # Prefix up to length 3

        # Prefixes of the word

        prefixes = []
        
        if len(token[0]) > 1:
            prefixes.append("PREF_" + token[0][:1])
        if len(token[0]) > 2:
            prefixes.append("PREF_" + token[0][:2])
        if len(token[0]) > 3:
            prefixes.append("PREF_" + token[0][:3])

        feature_list[1].append(prefixes)

        #POS

        feature_list[1].append("POS_" + token[1])

        # Word length

        feature_list[1].append("WORD_LENGTH_" + str(len(token[0])))

        # Stop word

        if token[0] in self.stop_words:
            feature_list[1].append("STOP_WORD")
        else:
            feature_list[1].append(None)

        # If capitalized
        if token[0][0].isupper():
            # Names

            if token[0] in self.names:
                feature_list[2].append("NAME")

            # Locations

            if token[0] in self.locations:
                feature_list[2].append("LOCATION")

        return feature_list


    

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\maxmg\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [5]:
def prep_train(corpus: List[List[tuple]], language = 'spanish') -> List[List[tuple]]:
    '''
    Preprocess the corpus for training

    :param corpus: list of list of tuples (word, tag)
    :type corpus: list(list(tuple(str, str)))
    :return: list of features
    :rtype: list(list(tuple(tuple(str, list(list(str))), str)))
    '''
    new_corpus = []
    for sentence in corpus:
        new_sentence = []
        for word_tuple in sentence:
            new_word_tuple = ((word_tuple[0], word_tuple[1]), word_tuple[2])
            new_sentence.append(new_word_tuple)
        new_corpus.append(new_sentence)
    new_corpus = FuncPreprocessing(language)(new_corpus)
    return new_corpus

In [6]:
def sep_token_tag(corpus: List[List[tuple]]) -> tuple[List[List[tuple]], List[List[str]]]:
    '''
    Separate the tokens and tags from the corpus
    
    :param corpus: list of list of tuples (word, tag)
    :type corpus: list(list(tuple(tuple(str, list(list(str))), str)))
    :return: tokens and tagsç
    :rtype: list(list(tuple)), list(list(str))
    '''
    tokens = [[word[0] for word in sentence] for sentence in corpus]
    tags = [[word[1] for word in sentence] for sentence in corpus]
    return tokens, tags

In [7]:
def prep_test(corpus: List[List[tuple]]) ->tuple[List[List[tuple]], List[List[str]]]:
    '''
    Preprocess the corpus for testing
    '''
    corpus_prep = prep_train(corpus)
    tokens, tags = sep_token_tag(corpus_prep)
    return tokens, tags

En les següents dues cel·les es mostra com eren les dades importades abans del preprocessing.

In [8]:
esp_train[0]

[('Melbourne', 'NP', 'B-LOC'),
 ('(', 'Fpa', 'O'),
 ('Australia', 'NP', 'B-LOC'),
 (')', 'Fpt', 'O'),
 (',', 'Fc', 'O'),
 ('25', 'Z', 'O'),
 ('may', 'NC', 'O'),
 ('(', 'Fpa', 'O'),
 ('EFE', 'NC', 'B-ORG'),
 (')', 'Fpt', 'O'),
 ('.', 'Fp', 'O')]

In [9]:
esp_testa[0]

[('Sao', 'NC', 'B-LOC'),
 ('Paulo', 'VMI', 'I-LOC'),
 ('(', 'Fpa', 'O'),
 ('Brasil', 'NC', 'B-LOC'),
 (')', 'Fpt', 'O'),
 (',', 'Fc', 'O'),
 ('23', 'Z', 'O'),
 ('may', 'NC', 'O'),
 ('(', 'Fpa', 'O'),
 ('EFECOM', 'NP', 'B-ORG'),
 (')', 'Fpt', 'O'),
 ('.', 'Fp', 'O')]

In [68]:
esp_train = prep_train(esp_train)
esp_testa_tokens, esp_testa_tags = prep_test(esp_testa)
esp_testb_tokens, esp_testb_tags = prep_test(esp_testb)

ned_train = prep_train(ned_train)
ned_testa_tokens, ned_testa_tags = prep_test(ned_testa)
ned_testb_tokens, ned_testb_tags = prep_test(ned_testb)

En les tres següents cel·les es mostra com queden les dades importades després del preprocessing. Es pot observar que ara el token va acompanyat de les llistes de funcions. També es van separar els tags de les particions de test i validació, per poder comparar-les amb les predites pels models.

In [69]:
esp_train[0]

[(('Melbourne',
   [['CAPITALIZATION', 'SUF_e', 'SUF_ne', 'SUF_rne', 'WORD_Melbourne'],
    [['PREF_M', 'PREF_Me', 'PREF_Mel'], 'POS_NP', 'WORD_LENGTH_9', None],
    ['LOCATION']]),
  'B-LOC'),
 (('(',
   [['PUNCTUATION', 'WORD_('], [[], 'POS_Fpa', 'WORD_LENGTH_1', None], []]),
  'O'),
 (('Australia',
   [['CAPITALIZATION', 'SUF_a', 'SUF_ia', 'SUF_lia', 'WORD_Australia'],
    [['PREF_A', 'PREF_Au', 'PREF_Aus'], 'POS_NP', 'WORD_LENGTH_9', None],
    ['LOCATION']]),
  'B-LOC'),
 ((')',
   [['PUNCTUATION', 'WORD_)'], [[], 'POS_Fpt', 'WORD_LENGTH_1', None], []]),
  'O'),
 ((',',
   [['PUNCTUATION', 'WORD_,'], [[], 'POS_Fc', 'WORD_LENGTH_1', None], []]),
  'O'),
 (('25',
   [['HAS_NUM', 'SUF_5', 'WORD_25'],
    [['PREF_2'], 'POS_Z', 'WORD_LENGTH_2', None],
    []]),
  'O'),
 (('may',
   [['SUF_y', 'SUF_ay', 'WORD_may'],
    [['PREF_m', 'PREF_ma'], 'POS_NC', 'WORD_LENGTH_3', None],
    []]),
  'O'),
 (('(',
   [['PUNCTUATION', 'WORD_('], [[], 'POS_Fpa', 'WORD_LENGTH_1', None], []]),
  'O'),


In [70]:
esp_testa_tokens[0]

[('Sao',
  [['CAPITALIZATION', 'SUF_o', 'SUF_ao', 'WORD_Sao'],
   [['PREF_S', 'PREF_Sa'], 'POS_NC', 'WORD_LENGTH_3', None],
   ['NAME']]),
 ('Paulo',
  [['CAPITALIZATION', 'SUF_o', 'SUF_lo', 'SUF_ulo', 'WORD_Paulo'],
   [['PREF_P', 'PREF_Pa', 'PREF_Pau'], 'POS_VMI', 'WORD_LENGTH_5', None],
   ['NAME', 'LOCATION']]),
 ('(',
  [['PUNCTUATION', 'WORD_('], [[], 'POS_Fpa', 'WORD_LENGTH_1', None], []]),
 ('Brasil',
  [['CAPITALIZATION', 'SUF_l', 'SUF_il', 'SUF_sil', 'WORD_Brasil'],
   [['PREF_B', 'PREF_Br', 'PREF_Bra'], 'POS_NC', 'WORD_LENGTH_6', None],
   ['LOCATION']]),
 (')',
  [['PUNCTUATION', 'WORD_)'], [[], 'POS_Fpt', 'WORD_LENGTH_1', None], []]),
 (',', [['PUNCTUATION', 'WORD_,'], [[], 'POS_Fc', 'WORD_LENGTH_1', None], []]),
 ('23',
  [['HAS_NUM', 'SUF_3', 'WORD_23'],
   [['PREF_2'], 'POS_Z', 'WORD_LENGTH_2', None],
   []]),
 ('may',
  [['SUF_y', 'SUF_ay', 'WORD_may'],
   [['PREF_m', 'PREF_ma'], 'POS_NC', 'WORD_LENGTH_3', None],
   []]),
 ('(',
  [['PUNCTUATION', 'WORD_('], [[], 'POS_

In [24]:
esp_testa_tags[0]

['B-LOC', 'I-LOC', 'O', 'B-LOC', 'O', 'O', 'O', 'O', 'O', 'B-ORG', 'O', 'O']

### Definició de GetFeatures

Per poder aprofitar aquest nou format de les dades, vam crear una classe que guardava com a variables de classe una sèrie de vectors i un booleà que indicaven quines funcions es volien o no utilitzar. El vector "features_vector" indica quina de les 4 primeres funcions extra s'utilitza. El booleà "lists" indica si s'utilitzen o no les llistes externes. I finalment, "word_vector" indica quin context de la paraula s'utilitza, permetent com a màxim dues paraules endavant i dues paraules enrere.

Les dues primeres estructures, "features_vector" i "lists", s'utilitzarien en aquest primer bloc de la pràctica, el de definir les funcions. I "word_vector" es va utilitzar en el segon bloc, el del context.

In [62]:
class GetFeatures:
            
    def __init__(self, features_vector = [0, 0, 0, 0], lists = False , word_vector = [0, 0, 1, 0, 0]):
            self.features_vector = features_vector
            self.lists = lists
            self.word_vector = word_vector

    def __call__(self, tokens: List[tuple[str, List[List[str]]]], idx: int) -> List[str]:
        '''
        Extract features from a token
        
        :param tokens: list of tuples (word, features)
        :type tokens: list(tuple(str, list(list(str))))
        :param idx: index of the token
        :type idx: int
        :return: list of features
        :rtype: list(str)
        '''
        feature_list = []

        if not tokens[idx][0]:
            return feature_list

        if self.word_vector[2] == 1:
            features = tokens[idx][1]

            for feat in features[0]:
                feature_list.append(feat)

            if self.features_vector[0] == 1:
                for pre in features[1][0]:
                    feature_list.append(pre)
            if self.features_vector[1] == 1:
                feature_list.append(features[1][1])
            if self.features_vector[2] == 1:
                feature_list.append(features[1][2])
            if self.features_vector[3] == 1:
                if features[1][3]:
                    feature_list.append(features[1][3])

            if self.lists:
                for feat in features[2]:
                    feature_list.append(feat)
            
        if idx > 1 and self.word_vector[0] == 1:
            features = tokens[idx - 2][1]

            for feat in features[0]:
                feature_list.append("2PREV" + feat)

            if self.features_vector[0] == 1:
                for pre in features[1][0]:
                    feature_list.append("2PREV" + pre)
            if self.features_vector[1] == 1:
                feature_list.append("2PREV" + features[1][1])
            if self.features_vector[2] == 1:
                feature_list.append("2PREV" + features[1][2])
            if self.features_vector[3] == 1:
                if features[1][3]:
                    feature_list.append("2PREV" + features[1][3])

            if self.lists:
                for feat in features[2]:
                    feature_list.append("2PREV" + feat)

        if idx > 0 and self.word_vector[1] == 1:
            features = tokens[idx - 1][1]

            for feat in features[0]:
                feature_list.append("PREV" + feat)

            if self.features_vector[0] == 1:
                for pre in features[1][0]:
                    feature_list.append("PREV" + pre)
            if self.features_vector[1] == 1:
                feature_list.append("PREV" + features[1][1])
            if self.features_vector[2] == 1:
                feature_list.append("PREV" + features[1][2])
            if self.features_vector[3] == 1:
                if features[1][3]:
                    feature_list.append("PREV" + features[1][3])

            if self.lists:
                for feat in features[2]:
                    feature_list.append("PREV" + feat)

        if idx < len(tokens) - 1 and self.word_vector[3] == 1:
            features = tokens[idx + 1][1]

            for feat in features[0]:
                feature_list.append("NEXT" + feat)

            if self.features_vector[0] == 1:
                for pre in features[1][0]:
                    feature_list.append("NEXT" + pre)
            if self.features_vector[1] == 1:
                feature_list.append("NEXT" + features[1][1])
            if self.features_vector[2] == 1:
                feature_list.append("NEXT" + features[1][2])
            if self.features_vector[3] == 1:
                if features[1][3]:
                    feature_list.append("NEXT" + features[1][3])

            if self.lists:
                for feat in features[2]:
                    feature_list.append("NEXT" + feat)

        if idx < len(tokens) - 2 and self.word_vector[4] == 1:
            features = tokens[idx + 2][1]

            for feat in features[0]:
                feature_list.append("2NEXT" + feat)

            if self.features_vector[0] == 1:
                for pre in features[1][0]:
                    feature_list.append("2NEXT" + pre)
            if self.features_vector[1] == 1:
                feature_list.append("2NEXT" + features[1][1])
            if self.features_vector[2] == 1:
                feature_list.append("2NEXT" + features[1][2])
            if self.features_vector[3] == 1:
                if features[1][3]:
                    feature_list.append("2NEXT" + features[1][3])

            if self.lists:
                for feat in features[2]:
                    feature_list.append("2NEXT" + feat)

        return feature_list
            

### Funció d'avaluació d'un model

Abans de començar a definir i experimentar amb les "features functions" i el context, era necessari establir un criteri de avaluació pels nostres models. En un primer lloc el vam establir per la codificació BIO, però més endavant el modificaríem per comparar les diferents codificacions.

Vam estar valorant diferents mètriques que ens aportessin informació real del rendiment del model. Al final vam decidir treballar amb dos enfocaments per poder valorar més correctament els models.

En primer lloc, vam considerar la "accuracy" balancejada, que ens servia per veure quantes etiquetes predites eren correctes. Per més endavant poder comparar entre codificacions, els "tags" real i predits es transformen, en primer lloc, a IO. També s'eliminen les categories d'entitats, i llavors es calcula la mètrica. Aquesta decisió la vam prendre amb la idea que aquesta mètrica valores essencialment la capacitat del model d'identificar entitats, no el seu tipus, ja que el tipus ja el tindríem més en compte a les altres mètriques (les explicades en el següent paràgraf). Vam decidir fer la "accuracy" balancejada pel fet que hi ha molta més quantitat de "O" en les frases. Com s'ha dit, aquesta mètrica ens va permetre valorar el rendiment del model a l'hora d'identificar "tags" individuals i sense importar el tipus. Sabíem que no era una mètrica que pogués, per ella sola, valorar correctament el model, però sí que cobria una part important.

Per complementar-la, vam decidir també fer un estudi de les entitats, no de les etiquetes. Amb diverses funcions vam aconseguir localitzar totes les entitats des de les etiquetes, i comparant les entitats reals amb les predites, vam poder extreure molta informació: quin percentatge d'entitats es detecten a la perfecció; quin percentatge de cada tipus d'entitats detecta correctament; i quantes entitats que realment no ho són "s'inventa" el model. Juntament amb la "accuracy" balancejada ens semblaven una sèrie de dades que permetien valorar a la perfecció el rendiment dels models.

Al final el criteri que vam seguir és donar-li més valor a detectar entitats perfectament, ja que és realment la tasca del model. A continuació es troben totes les funcions que conformen la funció de avaluació final.

In [95]:
def sent_tags_to_IO(sent_tags):
    '''
    Convert sentence tags to IO format
    
    :param sent_tags: list of tags
    :type sent_tags: list(str)
    :return: list of IO tags
    :rtype: list(str)
    '''
    io_sent_tags = []
    for sent in sent_tags:
        tags = [re.sub(r'\b[BES]-', 'I-', tag) for tag in sent]
        io_sent_tags.append(tags)
    return io_sent_tags

In [16]:
def entity_finder(sent_tags):
    '''
    Find entities in a sentence

    :param sent_tags: list of tags
    :type sent_tags: list(str)
    :return: list of entities
    :rtype: list(tuple(str, tuple(int, int)))
    '''
    sent_tags_io = sent_tags_to_IO(sent_tags)

    entities = []
    for sent in sent_tags_io:
        sent_entities = []
        last = sent[0]
        ini = None
        end = None
        type_ent = None
        if last[0] == 'I':
            ini = 0
            end = 0
            type_ent = last[2:]
        for i in range(1, len(sent)):
            tag = sent[i]
            if tag[0] == 'I':
                if last == 'O':
                    ini = i
                    end = i
                    type_ent = tag[2:]
                if last[0] == 'I':
                    end = i
                last = tag
            else:
                if last[0] == 'I':
                    sent_entities.append((type_ent, (ini, end)))
                    ini = None
                    end = None
                    type_ent = None
                last = tag
        if ini:
            sent_entities.append((type_ent, (ini, end)))
        entities.append(sent_entities)

    return entities

In [145]:
from sklearn.metrics import balanced_accuracy_score
from collections import Counter

def evaluate_tagger_performance(sent_real, sent_pred, features = [0,0,0,0], errors = False):
    '''
    Evaluate the performance of a tagger

    :param sent_real: list of real tags
    :type sent_real: list(list(str))
    :param sent_pred: list of predicted tags
    :type sent_pred: list(list(str))
    :param features: list of features to use
    :type features: list(int)
    :param errors: return errors
    :type errors: bool
    '''

    info = {'Codification': features, 'Balanced accuracy': 0, 'Total entities': 0, 'Entities correct': 0, 'LOC correct': 0, 'MISC correct': 0, 'ORG correct': 0, 'PER correct': 0, 'Entities invented': 0}
    
    sent_io_real = sent_tags_to_IO(sent_real)
    sent_io_pred = sent_tags_to_IO(sent_pred)

    def join_sent_tags(sent_tags):
        return [tag[0] for sent in sent_tags for tag in sent]

    # Calcula la precisión balanceada
    info['Balanced accuracy'] = balanced_accuracy_score(join_sent_tags(sent_io_real), join_sent_tags(sent_io_pred))

    real_entities = entity_finder(sent_io_real)
    pred_entities = entity_finder(sent_io_pred)

    total_ent = 0
    total_loc = 0
    total_misc = 0
    total_org = 0
    total_per = 0

    for sent in real_entities:
        for entity in sent:
            total_ent += 1
            if entity[0] == 'LOC':
                total_loc += 1
            if entity[0] == 'MISC':
                total_misc += 1
            if entity[0] =='ORG':
                total_org += 1
            if entity[0] == 'PER':
                total_per += 1

    info['Total entities'] = total_ent

    good_ent = 0
    good_loc = 0
    good_misc = 0
    good_org = 0
    good_per = 0
    invented_ent = 0

    for i in range(0, len(real_entities)):
        for entity in pred_entities[i]:
            if entity in real_entities[i]:
                good_ent += 1
                if entity[0] == 'LOC':
                    good_loc += 1
                if entity[0] == 'MISC':
                    good_misc += 1
                if entity[0] =='ORG':
                    good_org += 1
                if entity[0] == 'PER':
                    good_per += 1
            else:
                invented_ent += 1

    info['Entities correct'] = good_ent/total_ent
    info['LOC correct'] = good_loc/total_loc
    info['MISC correct'] = good_misc/total_misc
    info['ORG correct'] = good_org/total_org
    info['PER correct'] = good_per/total_per

    info['Entities invented'] = invented_ent

    if errors:
        errors = []
        invented = []
        for i in range(0, len(real_entities)):
            for entity in pred_entities[i]:
                if entity not in real_entities[i]:
                    errors.append((i, entity))
        for i in range(0, len(pred_entities)):
            for entity in real_entities[i]:
                if entity not in pred_entities[i]:
                    invented.append((i, entity))
        return info, errors, invented
                
    return info


### Grid search per definir les quatre primeres funcions extres

Un cop definit un criteri d'avaluació, vam decidir fer un "grid search" per seleccionar quines de les quatre primeres funcions extres utilitzaríem. Un petit recordatori de les quatre funcions:

1 - Prefixos
2 - Pos-Tag
3 - Longitud
4 - Stop-Word

Gràcies a l'eficiència i flexibilitat que ens proporcionava el "preprocessing" fet i la classe "GetFeatures" vam decidir provar totes les combinacions d'aquestes funcions possibles, i crear una taula per cada idioma amb els resultats. Després analitzaríem les mètriques i decidiríem quina combinació prendre, encara que esperàvem que en els dos casos fos la combinació de les quatre "feature functions".

Vam crear una funció que fes aquest "grid search", i la vam aplicar als dos idiomes.

In [79]:
import pandas as pd
import itertools
import nltk

def features_grid_search(train, val_tokens, val_tags):
    '''
    Perform a grid search over the features
    '''
    feature_combinations = list(itertools.product([0, 1], repeat=4))

    results = pd.DataFrame(columns=['Features', 'Balanced accuracy', 'Total entities', 'Entities correct', 'LOC correct', 'MISC correct', 'ORG correct', 'PER correct', 'Entities invented'])

    # Para cada combinación de características
    for features in feature_combinations:
        model = nltk.tag.CRFTagger(feature_func=GetFeatures(features_vector=features))
        model.train(train, 'crfTagger.mdl')
        prediction = model.tag_sents(val_tokens)

        _, prediction_tags = sep_token_tag(prediction)

        feat_results = evaluate_tagger_performance(val_tags, prediction_tags, features)

        results = pd.concat([results, pd.DataFrame([feat_results])], ignore_index=True)

    return results

### Anàlisi de "feature functions" en el model de l'idioma espanyol

In [ ]:
results = features_grid_search(esp_train, esp_testa_tokens, esp_testa_tags)

In [29]:
results

,Features,Balanced accuracy,Total entities,Entities correct,LOC correct,MISC correct,ORG correct,PER correct,Entities invented
0,"(0, 0, 0, 0)",0.961174,4274,0.677585,0.741834,0.471655,0.673977,0.707602,1165
1,"(0, 0, 0, 1)",0.960965,4274,0.674310,0.744995,0.469388,0.666864,0.704261,1176
2,"(0, 0, 1, 0)",0.961637,4274,0.674310,0.742887,0.458050,0.656194,0.725146,1188
3,"(0, 0, 1, 1)",0.961802,4274,0.674544,0.742887,0.458050,0.656787,0.725146,1185
4,"(0, 1, 0, 0)",0.966648,4274,0.691390,0.763962,0.467120,0.684647,0.725982,1168
5,"(0, 1, 0, 1)",0.966934,4274,0.690688,0.765016,0.462585,0.682276,0.727652,1173
6,"(0, 1, 1, 0)",0.966351,4274,0.690688,0.749210,0.460317,0.682869,0.740184,1180
7,"(0, 1, 1, 1)",0.966626,4274,0.691858,0.749210,0.464853,0.683462,0.741855,1175
8,"(1, 0, 0, 0)",0.962893,4274,0.692326,0.775553,0.462585,0.676941,0.732665,1135
9,"(1, 0, 0, 1)",0.962232,4274,0.690454,0.772392,0.453515,0.673977,0.736007,1140


Un cop creada la taula de resultats de totes les combinacions de l'idioma espanyol, vam procedir a fer una anàlisi dels resultats.

Primerament, observant la "balanced accuracy" vam poder comprovar que en tots el casos el model ubica bastant bé on hi ha entitats, ja que tots els resultats són majors al 0.96. Però en visualitzar les altres mètriques vam veure que no són tan bons a l'hora de detectar perfectament la ubicació i el tipus de l'entitat. Com que vam imaginar que seria una cosa recurrent fins i tot en el model final, vam decidir que més endavant analitzaríem alguns casos reals per entendre en quin sentit perdia tanta precisió en el tipus i en quins casos s'estava equivocant.

Seguint en el mateix tema, també vam observar que el marge de millora es situava en el fet de detectar correctament les entitats i el seu tipus. Mentres en la "accuracy" la millora màxima és d'aproximadament 0.007, en el percentatge d'entitats correctes la millora màxima és de més o menys 0.03. Això ens va fer pensar que les noves "feature functions" estaven ajudant el model a detectar amb més precisió el tipus i posició exacte de l'entitat.

Ja entrant més específicament en les combinacions, primer vam observar com milloraven les funcions de forma individual, és a dir, quan només s'activaven elles. Ens vam centrar a veure com millorava el percentatge d'entitats correctes. Les funcions que més milloraven el model eren els prefixos i el Pos-Tag. En canvi, les altres dues, no només no milloraven, sinó que feien lleugerament pitjor al model.

Ja fixant-nos amb les combinacions, vam veure que la tercera funció, la longitud, si aportava un millora en els models amb més funcions actives, però les Stop-Words no estaven aportant pràcticament res, així que vam decidir no agafar-la. Per tant, la nostra combinació de funcions extres per l'espanyol va ser la [1, 1, 1, 0].

També per comentar-ho, tots els models en general inventen bastantes entitats, però sí que, normalment, a mesura que s'activen funcions es va reduint el nombre de entitats inventades. En general, tots els tipus milloren o empitjoren en relació amb el total d'entitats, no sembla que hi hagi funciono que potenciïn sobretot un tipus d'entitat. Sí que es pot apreciar clarament que el tipus MISC és el que pitjor es detecta. Al final té sentit, ja que és el tipus més obert, que cobreix les restes.

### Anàlisi de "feature functions" en el model de l'idioma holandés

In [ ]:
results2 = features_grid_search(ned_train, ned_testa_tokens, ned_testa_tags)

In [31]:
results2

,Features,Balanced accuracy,Total entities,Entities correct,LOC correct,MISC correct,ORG correct,PER correct,Entities invented
0,"(0, 0, 0, 0)",0.938590,2276,0.576450,0.570787,0.580451,0.445596,0.705281,772
1,"(0, 0, 0, 1)",0.938291,2276,0.575132,0.570787,0.578947,0.445596,0.701874,773
2,"(0, 0, 1, 0)",0.949600,2276,0.592707,0.602247,0.615038,0.433506,0.717206,834
3,"(0, 0, 1, 1)",0.950199,2276,0.590510,0.597753,0.615038,0.426598,0.718910,833
4,"(0, 1, 0, 0)",0.960987,2276,0.609842,0.608989,0.633083,0.442142,0.749574,751
5,"(0, 1, 0, 1)",0.960448,2276,0.609842,0.608989,0.634586,0.442142,0.747871,747
6,"(0, 1, 1, 0)",0.964428,2276,0.614675,0.613483,0.654135,0.430052,0.752981,786
7,"(0, 1, 1, 1)",0.964832,2276,0.615114,0.613483,0.654135,0.430052,0.754685,785
8,"(1, 0, 0, 0)",0.939034,2276,0.618190,0.638202,0.590977,0.504318,0.746167,679
9,"(1, 0, 0, 1)",0.941577,2276,0.617311,0.638202,0.590977,0.504318,0.742760,700


Després d'analitzar els resultats en l'idioma espanyol, vam procedir a fer el mateix amb l'holandès. I ràpidament vam veure diferències i similituds importants.

El que vam veure més ràpidament i més ens va sobtar va ser la precisió en els diferents tipus d'entitat. MISC, que en l'espanyol era clarament la pitjor, ara es prediu molt millor, quedant per sobre de LOC i ORG, sent aquesta última la pitjor amb diferència.

També vam observar clarament dues coses. La primera és que, igual que abans, la "accuracy" balancejada era molt alta i la poca precisió residia en detectar a la perfecció les entitats. Però, en segon lloc, vam veure que ara el rang de millor en ambdós casos era molt superior, sent aproximadament com a màxim de 0.03 i 0.08 respectivament. Per tant, ara les funcions estaven millorant més els models.

Quant a les funcions individuals, els resultats van ser semblant que a l'espanyol. Ara, individualment, la funció 3 si millora una mica, però la 4 segueix abans que igual. La 1 i la 2 segueixen sent les que tenen més impacte.

Finalment, en les combinacions vam poder extreure les mateixes conclusions. La longitud si aporta però les Stop-Words no. Així que vam decidir prendre la mateixa combinació que amb l'espanyol, el vector [1, 1, 1, 0].

Com a comentari final, en aquest idioma també hi ha un alt nombre d'entitats inventades, que baixen a l'hora d'afegir funcions. Quant a proporció amb el total d'entitats que hi ha, el resultat és semblant a l'idioma espanyol.

### Prova d'afegir llistes externes de noms i localitzacions

Per acabar de tancar aquest bloc, vam decidir afegir també la cinquena funció extra, la relacionada amb llistes. Per provar simplement vam comparar els dos models seleccionats a l'apartat anterior amb ells mateixos però afegint les llistes de noms i localitzacions. Aquestes funcions ja venien calculades al preprocessament, però ara expliquem quines llistes vam utilitzar.

Vam decidir afegir noms i localitzacions, ja que ens semblava el més recurrent i típic i fàcil de repetir-se.

Pels noms vam buscar noms espanyols i holandesos. Vam trobar dos repositoris de Git-Hub on hi havia arxius amb noms, en un cas espanyols, i en l'altre cas holandesos. A continuació els enllaços:

Espanyol ---> https://github.com/olea/lemarios/blob/master/nombres-propios-es.txt

Holandès ---> https://github.com/digitalheir/family-names-in-the-netherlands/blob/master/family_names_freq_5_or_more.lst

Per les localitzacions també vam usar un repositori que contenia noms de països i ciutats en format JSON. A continuació l'enllaç:

https://github.com/millan2993/countries/tree/master/json

Tots els arxius utilitzats es troben a l'arxiu comprimit entregat.

Per provar i fer la comparació dels models es va crear una funció que executava el model amb llistes i sense. Després vam analitzar els resultats per cada idioma.

In [80]:
def list_comparation(train, val_tokens, val_tags):
    '''
    Compare the performance of the tagger with and without lists
    '''
    results = pd.DataFrame(columns=['Lists', 'Balanced accuracy', 'Total entities', 'Entities correct', 'LOC correct', 'MISC correct', 'ORG correct', 'PER correct', 'Entities invented'])

    model = nltk.tag.CRFTagger(feature_func=GetFeatures(features_vector=[1, 1, 1, 0], lists=False))
    model.train(train, 'crfTagger.mdl')
    prediction = model.tag_sents(val_tokens)

    _, prediction_tags = sep_token_tag(prediction)

    feat_results = evaluate_tagger_performance(val_tags, prediction_tags, "No lists")

    results = pd.concat([results, pd.DataFrame([feat_results])], ignore_index=True)

    model = nltk.tag.CRFTagger(feature_func=GetFeatures(features_vector=[1, 1, 1, 0], lists=True))
    model.train(train, 'crfTagger.mdl')
    prediction = model.tag_sents(val_tokens)

    _, prediction_tags = sep_token_tag(prediction)

    feat_results = evaluate_tagger_performance(val_tags, prediction_tags, "Lists")

    results = pd.concat([results, pd.DataFrame([feat_results])], ignore_index=True)

    return results

### Anàlisi de l'afegit de llistes externes als dos models

In [81]:
results3 = list_comparation(esp_train, esp_testa_tokens, esp_testa_tags)
results4 = list_comparation(ned_train, ned_testa_tokens, ned_testa_tags)

In [82]:
results3

,Lists,Balanced accuracy,Total entities,Entities correct,LOC correct,MISC correct,ORG correct,PER correct,Entities invented
0,No lists,0.967353,4274,0.704960,0.777661,0.478458,0.690575,0.751044,1125
1,Lists,0.966913,4274,0.711511,0.793467,0.476190,0.688204,0.766082,1082


In [83]:
results4

,Lists,Balanced accuracy,Total entities,Entities correct,LOC correct,MISC correct,ORG correct,PER correct,Entities invented
0,No lists,0.963677,2276,0.653339,0.624719,0.663158,0.530225,0.785349,680
1,Lists,0.960402,2276,0.661248,0.696629,0.652632,0.528497,0.775128,641


En veure els resultats vam comprovar que en els dos casos disminueix lleugerament la "balanced accuracy", però hi ha una important millora en les entitats predites correctament.

En l'espanyol, els resultats dels tipus tenen molts a veure amb les funcions de les llistes que s'han incorporat. Hi ha una millora al trobar localitzacions i persones, i una petita pèrdua de rendiment en els tipus ORG i MISC. A l'haver incorporat llistes justament de persones i llocs, té sentit que hi hagi una millora. I també té sentit que hi hagi un empitjorament en els altres tipus, perquè segurament les llistes han provocat que organitzacions i altres esdeveniments que abans es predeien bé ara es considerin persones o indrets, perquè alguna de les seves parts activen la funció de nom o localització.

En l'holandès passa una cosa una mica més curiosa. Tota la millora es troba en les localitzacions, amb un augment de més 0.07. En canvi, tots els altres tipus empitjoren. Per veure si es devia a algun tipus de desbalanceig vam contar el nombre d'entitats de cada tipus en la partició de validació holandesa.

In [87]:
def entity_type_counter(sent_tags):
    '''
    Count the number of entities of each type
    '''
    entities = entity_finder(sent_tags)

    loc_counter = 0
    misc_counter = 0
    org_counter = 0
    per_counter = 0
    
    for sent in entities:
        for entity in sent:
            if entity[0] == 'LOC':
                loc_counter += 1
            if entity[0] == 'MISC':
                misc_counter += 1
            if entity[0] == 'ORG':
                org_counter += 1
            if entity[0] == 'PER':
                per_counter += 1

    print("LOC: ", loc_counter)
    print("MISC: ", misc_counter)
    print("ORG: ", org_counter)
    print("PER: ", per_counter) 

entity_type_counter(ned_testa_tags)

LOC:  445
MISC:  665
ORG:  579
PER:  587


Vam observar que localització era el tipus amb menys presència, així que igual aquest fet tenia alguna cosa a veure. A l'haver-hi menys casos, igual en prediu més i se centra més en aquest tipus empitjorant els altres tres.

En tot cas, vam considerar que els models amb les llistes eren millors que sense elles. A part que la capacitat de trobar les entitats correctament era major, també reduïa significativament el nombre d'entitats inventades.

Així que, finalment, de les cinc noves "feature functions" proposades ens vam quedar amb quatre:

- Prefixos

- Pos-Tag

- Longitud

- Llistes de noms i localitzacions

Aquestes funcions més les bàsiques serien les usades en l'experimentació del context del següent bloc, i en els models finals.

## 2 - Experimentació de context

Un cop decidides les "feature functions" per cada idioma, vam passar al bloc 2, l'experimentació de context. Fins ara només se li donava al model informació del token a predir. Ara, ajustant el "word_vector" de GetFeatures podem donar-li al model context de fins a dues paraules abans i després de l'actual.

Vam decidir provar totes les combinacions lògiques del vector. Això vol dir que sempre es dona informació de la paraula actual i que no es dona informació de dues paraules més enllà si no es dona informació de la paraula entremig. Al final les combinacions són les següents:

- Paraula actual

- Paraula anterior, paraula actual

- Paraula dos cops anterior, paraula anterior, paraula actual

- Paraula actual, paraula següent

- Paraula actual, paraula següent, paraula dos cops següent

- Paraula anterior, paraula actual, paraula següent

- Paraula anterior, paraula actual, paraula següent, paraula dos cops següent

- Paraula dos cops anterior, paraula anterior, paraula actual, paraula següent

- Paraula dos cops anterior, paraula anterior, paraula actual, paraula següent, paraula dos cops següent

Vam crear una funció de "grid search" per provar totes les combinacions i vam aplicar-la als dos idiomes. Després vam analitzar els resultats per decidir quin context prendre en cada cas.

In [93]:
def word_vector_search(train, val_tokens, val_tags):
    '''
    Perform a grid search over the word vector
    '''
    word_combinations = [[0, 0, 1, 0, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [0, 0, 1, 1, 0], [0, 0, 1, 1, 1], [0, 1, 1, 1, 0], [1, 1, 1, 1, 0], [0, 1, 1, 1, 1], [1, 1, 1, 1, 1]]

    results = pd.DataFrame(columns=['Word vector', 'Balanced accuracy', 'Total entities', 'Entities correct', 'LOC correct', 'MISC correct', 'ORG correct', 'PER correct', 'Entities invented'])

    # Para cada combinación de características
    for word_vector in word_combinations:
        model = nltk.tag.CRFTagger(feature_func=GetFeatures(features_vector=[1, 1, 1, 0], lists=True, word_vector=word_vector))
        model.train(train, 'crfTagger.mdl')
        prediction = model.tag_sents(val_tokens)

        _, prediction_tags = sep_token_tag(prediction)

        feat_results = evaluate_tagger_performance(val_tags, prediction_tags, word_vector)

        results = pd.concat([results, pd.DataFrame([feat_results])], ignore_index=True)

    return results

### Anàlisi de context en el model de l'idioma espanyol

In [94]:
results5 = word_vector_search(esp_train, esp_testa_tokens, esp_testa_tags)

In [102]:
results5

,Word vector,Balanced accuracy,Total entities,Entities correct,LOC correct,MISC correct,ORG correct,PER correct,Entities invented
0,"[0, 0, 1, 0, 0]",0.966913,4274,0.711511,0.793467,0.476190,0.688204,0.766082,1082
1,"[0, 1, 1, 0, 0]",0.968886,4274,0.747075,0.808219,0.507937,0.730883,0.809524,995
2,"[1, 1, 1, 0, 0]",0.968291,4274,0.743332,0.805058,0.478458,0.736811,0.801170,973
3,"[0, 0, 1, 1, 0]",0.969634,4274,0.716893,0.793467,0.487528,0.700652,0.763576,1102
4,"[0, 0, 1, 1, 1]",0.967939,4274,0.713383,0.789252,0.430839,0.724956,0.741019,1147
5,"[0, 1, 1, 1, 0]",0.969691,4274,0.748947,0.802950,0.503401,0.749259,0.796157,980
6,"[1, 1, 1, 1, 0]",0.969250,4274,0.746373,0.804004,0.487528,0.748074,0.793651,986
7,"[0, 1, 1, 1, 1]",0.967246,4274,0.743566,0.812434,0.464853,0.751037,0.781119,1000
8,"[1, 1, 1, 1, 1]",0.966442,4274,0.738886,0.801897,0.455782,0.748666,0.779449,988


Els resultats van ser semblants als esperats. Afegir funcions del context millora el rendiment del model. Però vam observar que donar massa context empitjorava una mica els resultats comparats amb quant es donava menys context.

Així doncs, el millor resultat el vam trobar en la següent combinació:

- Paraula anterior, paraula actual, paraula següent

Millorava tots els tipus d'entitats i també disminuïa bastant el nombre d'entitats inventades. Així que el nostre model final de la llengua espanyola tenia els següents paràmetres:

- Features_vector = [1, 1, 1, 0]

- Lists = True

- Word_vector = [0, 1, 1, 1, 0]

### Anàlisi de context en el model de l'idioma holandès

In [103]:
results6 = word_vector_search(ned_train, ned_testa_tokens, ned_testa_tags)

In [116]:
results6 

,Word vector,Balanced accuracy,Total entities,Entities correct,LOC correct,MISC correct,ORG correct,PER correct,Entities invented
0,"[0, 0, 1, 0, 0]",0.960402,2276,0.661248,0.696629,0.652632,0.528497,0.775128,641
1,"[0, 1, 1, 0, 0]",0.966752,2276,0.708260,0.752809,0.711278,0.538860,0.838160,573
2,"[1, 1, 1, 0, 0]",0.966723,2276,0.699912,0.746067,0.714286,0.533679,0.812606,592
3,"[0, 0, 1, 1, 0]",0.960341,2276,0.659490,0.687640,0.678195,0.525043,0.749574,662
4,"[0, 0, 1, 1, 1]",0.960488,2276,0.656854,0.689888,0.688722,0.511226,0.739353,675
5,"[0, 1, 1, 1, 0]",0.967169,2276,0.709578,0.775281,0.715789,0.540587,0.819421,574
6,"[1, 1, 1, 1, 0]",0.965881,2276,0.705185,0.761798,0.714286,0.547496,0.807496,585
7,"[0, 1, 1, 1, 1]",0.968334,2276,0.706503,0.746067,0.721805,0.547496,0.816014,591
8,"[1, 1, 1, 1, 1]",0.966226,2276,0.706503,0.743820,0.726316,0.552677,0.807496,589


En el cas del holandès vam veure un resultat molt semblant. El context millora el rendiment del model, però si es dona massa context comença a anar pitjor. 

Així que el resultat del context va tornar a ser el mateix:

- Paraula anterior, paraula actual, paraula següent

També millora totes els tipus de entitats i torna a disminuir el nombre d'entitats inventades. Així que el nostre model final de la llengua holandesa va tenir els següents paràmetres:

- Features_vector = [1, 1, 1, 0]

- Lists = True

- Word_vector = [0, 1, 1, 1, 0]

## 3 - Experimentació de les codificacions

Un cop vam tenir els dos models finals creats, vam provar-los en diferents codificacions per escollir-ne una. Vam decidir provar sobre les següents codificacions:

- IO

- BIO

- BIOE

- BIOS

- BIOES

El primer que vam fer és crear diferents funcions que generessin les noves codificacions. Es troben a continuació.

In [104]:
def train_to_IO(train):
    '''
    Convert the train to IO format
    '''
    train_io = []
    for sent in train:
        sent_io = []
        for word in sent:
            new_tag = re.sub(r'\bB-', 'I-', word[1])
            sent_io.append((word[0], new_tag))
        train_io.append(sent_io)
    return train_io

In [135]:
esp_train_IO = train_to_IO(esp_train)
ned_train_IO = train_to_IO(ned_train)

In [114]:
def train_to_BIOE(train):
    '''
    Convert the train to BIOE format
    '''
    train_bioe = []
    for sent in train:
        sent_bioe = []
        for i in range(len(sent)):
            tag = sent[i][1]
            if tag[0] == 'I' and (i == len(sent) - 1 or sent[i + 1][1][0] != 'I'):
                tag = re.sub(r'\bI-', 'E-', tag)
            sent_bioe.append((sent[i][0], tag))
        train_bioe.append(sent_bioe)
    return train_bioe

In [136]:
esp_train_BIOE = train_to_BIOE(esp_train)
ned_train_BIOE = train_to_BIOE(ned_train)

In [128]:
def train_to_BIOS(train):
    '''
    Convert the train to BIOS format
    '''
    train_bios = []
    for sent in train:
        sent_bios = []
        for i in range(len(sent)):
            tag = sent[i][1]
            if tag[0] == 'B' and (i == len(sent) - 1 or sent[i + 1][1][0] == 'O'):
                tag = re.sub(r'\bB-', 'S-', tag)
            sent_bios.append((sent[i][0], tag))
        train_bios.append(sent_bios)
    return train_bios

In [137]:
esp_train_BIOS = train_to_BIOS(esp_train)
ned_train_BIOS = train_to_BIOS(ned_train)

In [129]:
def train_to_BIOES(train):
    '''
    Convert the train to BIOES format
    '''
    train_bios = train_to_BIOS(train)
    train_bioes = train_to_BIOE(train_bios)
    return train_bioes

In [138]:
esp_train_BIOES = train_to_BIOES(esp_train)
ned_train_BIOES = train_to_BIOES(ned_train)

Un cop creades totes les noves particions de train amb les 5 codificacions, vam crear una funció de "grid search" per provar les diferents codificacions en cada idioma, i veure els resultats en una taula.

La funció es troba a continuació.

In [140]:
cod_train_esp = [esp_train, esp_train_IO, esp_train_BIOE, esp_train_BIOS, esp_train_BIOES]
cod_train_ned = [ned_train, ned_train_IO, ned_train_BIOE, ned_train_BIOS, ned_train_BIOES]
cod = ['BIO','IO', 'BIOE', 'BIOS', 'BIOES']

def cod_grid_search(trains, val_tokens, val_tags, cod):
    '''
    Perform a grid search over the codification
    '''
    results = pd.DataFrame(columns=['Codification', 'Balanced accuracy', 'Total entities', 'Entities correct', 'LOC correct', 'MISC correct', 'ORG correct', 'PER correct', 'Entities invented'])

    # Para cada combinación de características
    for i in range(len(trains)):
        model = nltk.tag.CRFTagger(feature_func=GetFeatures(features_vector=[1, 1, 1, 0], lists=True, word_vector=[0, 1, 1, 1, 0]))
        model.train(trains[i], 'crfTagger.mdl')
        prediction = model.tag_sents(val_tokens)

        _, prediction_tags = sep_token_tag(prediction)

        feat_results = evaluate_tagger_performance(val_tags, prediction_tags, cod[i])

        results = pd.concat([results, pd.DataFrame([feat_results])], ignore_index=True)

    return results

### Anàlisi de codificacions en el model espanyol

In [141]:
results7 = cod_grid_search(cod_train_esp, esp_testa_tokens, esp_testa_tags, cod)

In [142]:
results7

,Codification,Balanced accuracy,Total entities,Entities correct,LOC correct,MISC correct,ORG correct,PER correct,Entities invented
0,BIO,0.969691,4274,0.748947,0.802950,0.503401,0.749259,0.796157,980
1,IO,0.969260,4274,0.742630,0.808219,0.485261,0.730883,0.802005,973
2,BIOE,0.969614,4274,0.747777,0.802950,0.501134,0.747481,0.795322,988
3,BIOS,0.971475,4274,0.745905,0.812434,0.510204,0.738589,0.790309,1002
4,BIOES,0.971310,4274,0.749181,0.813488,0.507937,0.750445,0.785297,1007


Després de provar les diferents codificacions en el model espanyol, vam observar que no hi havia pràcticament diferència entres elles.

La codificació que millor "balanced accuracy" i precisió en detectar entitats donava era la codificació BIOES, però amb una diferència mínima respecte a la codificació original. A part, la codificació BIOES augmentava una mica el nombre d'entitats inventades, així que vam decidir mantenir la codificació original, la BIO.

### Anàlisi de codificacions en el model holandès

In [143]:
results8 = cod_grid_search(cod_train_ned, ned_testa_tokens, ned_testa_tags, cod)

In [144]:
results8

,Codification,Balanced accuracy,Total entities,Entities correct,LOC correct,MISC correct,ORG correct,PER correct,Entities invented
0,BIO,0.967169,2276,0.709578,0.775281,0.715789,0.540587,0.819421,574
1,IO,0.966092,2276,0.703866,0.766292,0.693233,0.561313,0.809199,597
2,BIOE,0.964893,2276,0.706942,0.759551,0.714286,0.535406,0.827939,573
3,BIOS,0.964026,2276,0.694640,0.746067,0.703759,0.518135,0.819421,602
4,BIOES,0.963874,2276,0.700351,0.748315,0.709774,0.526770,0.824532,595


En el cas de l'holandès el resultat no va ser gaire diferent. El fet de canviar la codificació no canviava gaire el resultat. De fet, en aquest cas, totes les codificacions empitjoraven respecte de l'original.

Així que no va haver-hi gaire dubte en aquest cas. Igual que en l'idioma espanyol, vam seleccionar com a definitiva la codificació BIO.

## 4 - Anàlisi dels models definitius

Un cop ja vam passar els primers tres blocs, vam trobar dos models definitius.

En el primer bloc vam seleccionar quines "feature functions" utilitzaríem. Al final van resultar ser en els dos casos les 5 funcions bàsiques, els prefixos, el Pos-Tag i la longitud.

En el segon bloc vam determinar quin context usar. Vam considerar en els dos idiomes el mateix context, la paraula anterior, actual i següent.

En el tercer i últim bloc d'experimentació vam experimentar amb diferents codificacions per escollir-ne una, i finalment vam comprovar que l'original era la millor en ambdós casos.

Un cop vam tenir els dos models definits els vam aplicar al test per veure els resultats, i també vam analitzar els errors per veure en quins casos no detectava entitats i en quins casos se les inventava.

### Anàlisi dels resultats i errors del model espanyol sobre el test

In [147]:
model_esp = nltk.tag.CRFTagger(feature_func=GetFeatures(features_vector=[1, 1, 1, 0], lists=True, word_vector=[0, 1, 1, 1, 0]))
model_esp.train(esp_train, 'crfTagger.mdl')
prediction_esp = model_esp.tag_sents(esp_testb_tokens)

_, prediction_esp_tags = sep_token_tag(prediction_esp)

results, errors = evaluate_tagger_performance(esp_testb_tags, prediction_esp_tags, features=[1,1,1,0], errors=True)

In [151]:
results

{'Codification': [1, 1, 1, 0],
 'Balanced accuracy': 0.9705465184033057,
 'Total entities': 3536,
 'Entities correct': 0.7864819004524887,
 'LOC correct': 0.7684407096171803,
 'MISC correct': 0.45722713864306785,
 'ORG correct': 0.8254649499284692,
 'PER correct': 0.8914835164835165,
 'Entities invented': 720}

Vam entrenar el model definitiu de l'espanyol amb tots els paràmetres seleccionats al llarg de la pràctica. I el vam aplicar a la partició de test.

Els resultats són molt bons. La "accuracy" balancejada és molt alta, el que significa que localitza molt bé les entitats. Quant a predir perfectament la posició i tipus de les entitats, prediu correctament quasi el 80% de les entitats. I prediu molt bé les persones, amb una precisió de quasi el 90%. El tipus pitjor predit és el de MISC, com ja passava a la validació.

Vam voler analitzar alguns casos en els quals el model fallava per poder fer una millor anàlisi d'aquest. Per fer-ho vam utilitzar el paràmetre errors de la funció evaluate_tagger_performance. A continuació vam analitzar alguns casos aleatoris.

In [162]:
import random
random.seed(21)
# Escoge un error aleatorio
error = random.choice(errors)

idx = error[0]
entity = error[1]

sentence_bad = esp_testb[idx]

sentence = [token[0] for token in sentence_bad]

print("Sentence: ", sentence)

print("Real tags:", [tag for tag in esp_testb_tags[idx]])

print("Predicted tags:", [tag[1] for tag in prediction_esp[idx]])

Sentence:  ['Vallehermoso', ',', 'la', 'única', 'inmobiliaria', 'española', 'que', 'forma', 'parte', 'del', 'índice', 'Ibex-35', ',', 'obtuvo', 'en', 'el', 'primer', 'trimestre', 'de', 'este', 'año', 'unos', 'beneficios', 'antes', 'de', 'impuestos', 'de', '4.711', 'millones', 'de', 'pesetas', ',', 'un', '32,5', 'por', 'ciento', 'más', 'que', 'en', 'el', 'ejercicio', 'precedente', '.']
Real tags: ['B-ORG', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-MISC', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
Predicted tags: ['B-LOC', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-MISC', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']


En aquest primer exemple es pot veure que l'errada està en la primera paraula. Realment és una organització, però es prediu com a localització. És un error que es pot entendre, ja que realment sembla el nom d'un lloc, i per tant no és una gran errada.

In [163]:
import random
random.seed(23)
# Escoge un error aleatorio
error = random.choice(errors)

idx = error[0]
entity = error[1]

sentence_bad = esp_testb[idx]

sentence = [token[0] for token in sentence_bad]

print("Sentence: ", sentence)

print("Real tags:", [tag for tag in esp_testb_tags[idx]])

print("Predicted tags:", [tag[1] for tag in prediction_esp[idx]])

Sentence:  ['La', 'Asociación', '"', 'Sancho', 'Ramírez', '"', 'de', 'Jaca', '(', 'Huesca', ')', 'ha', 'puesto', 'en', 'marcha', 'un', 'concurso', 'de', 'fotografía', 'y', 'dibujo', 'sobre', 'la', '"', 'Arquitectura', 'popular', 'en', 'el', 'Pirineo', 'Aragonés', '"', ',', 'con', 'el', 'objetivo', 'de', 'llamar', 'la', 'atención', 'sobre', 'el', 'patrimonio', 'para', 'que', 'sea', 'protegido', ',', 'conservado', 'y', 'difundido', '.']
Real tags: ['O', 'B-ORG', 'I-ORG', 'I-ORG', 'I-ORG', 'I-ORG', 'O', 'B-LOC', 'O', 'B-LOC', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-MISC', 'I-MISC', 'I-MISC', 'I-MISC', 'I-MISC', 'I-MISC', 'I-MISC', 'I-MISC', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
Predicted tags: ['O', 'B-ORG', 'I-ORG', 'I-ORG', 'I-ORG', 'O', 'O', 'B-LOC', 'O', 'B-LOC', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-MISC', 'O', 'O', 'O', 'B-MISC', 'I-MISC', 'O', 'O', 'O', 'O', 

Aquest segon cas és molt més complex. Les entitats són molt més llargues i això provoca que el model no les acabi de detectar totes correctament. Així que el model pot fallar en casos on hi ha entitats extenses.

### Conclusions del model espanyol

Després d'aplicar el model al test vam obtenir uns bons resultats. 4 de cada 5 entitats són detectades perfectament, i pràcticament totes les entitats són ubicades, encara que no amb el tipus correcte.

Pel vist al test, el model és especialment bo per predir persones, i també dona un molt bon resultat en localitzacions i organitzacions. El tipus MISC és el que més costa de detectar correctament, i la precisió és del 50%.

Amb l'anàlisi d'errors vam poder veure que el model fallava en casos realment complicats, amb dues dinàmiques principals:

-Paraules estranyes que es prediuen com un tipus que no són, com per exemple l'error 1

-Entitats extenses que no s'acaben de predir correctament, com per exemple l'error 2

Al llarg de la pràctica hem anat definint les millors opcions a través de la validació, i creiem després dels tres blocs ha quedat un molt bon model final, que ubica pràcticament totes les entitats, i que a vegades falla a l'hora de precisar el tipus.

### Anàlisi dels resultats i errors del model holandès sobre el test

In [150]:
model_ned = nltk.tag.CRFTagger(feature_func=GetFeatures(features_vector=[1, 1, 1, 0], lists=True, word_vector=[0, 1, 1, 1, 0]))
model_ned.train(ned_train, 'crfTagger.mdl')
prediction_ned = model_ned.tag_sents(ned_testb_tokens)

_, prediction_ned_tags = sep_token_tag(prediction_ned)

results2, errors2 = evaluate_tagger_performance(ned_testb_tags, prediction_ned_tags, features=[1,1,1,0], errors=True)

In [164]:
results2

{'Codification': [1, 1, 1, 0],
 'Balanced accuracy': 0.9652629808175924,
 'Total entities': 3409,
 'Entities correct': 0.7450865356409504,
 'LOC correct': 0.8066298342541437,
 'MISC correct': 0.6887417218543046,
 'ORG correct': 0.5904109589041096,
 'PER correct': 0.8875278396436526,
 'Entities invented': 746}

Vam entrenar el model definitiu holandès amb tots els paràmetres definits durant l'experimentació, i el vam aplicar al test.

Els resultats ens van sorprendre bastant, ja que eren superiors al vist en la validació. La "accuracy" balancejada és molt alta, el que significa que s'ubiquen molt bé les entitats. El percentatge d'entitats predites perfectament és del 75%, una mica inferior que en el model espanyol. Un altre cop el test ens diu que és un bon model per predir persones. Igual que l'espanyol també té una alta precisió en les localitzacions, de fet més. Però on hi ha molta diferència és a ORG i MISC. En aquest idioma MISC es prediu molt millor que a l'espanyol, i amb les organitzacions passa el contrari, es prediuen molt pitjor que en l'espanyol.

També vam fer una anàlisi d'errors aleatoris en aquest idioma.

In [167]:
import random
random.seed(23)
# Escoge un error aleatorio
error = random.choice(errors2)

idx = error[0]
entity = error[1]

sentence_bad = ned_testb[idx]

sentence = [token[0] for token in sentence_bad]

print("Sentence: ", sentence)

print("Real tags:", [tag for tag in ned_testb_tags[idx]])

print("Predicted tags:", [tag[1] for tag in prediction_ned[idx]])

Sentence:  ['Transparency', 'International', '(', 'TI', ')', ',', 'de', 'onderhand', 'in', '75', 'naties', 'vertegenwoordigde', 'Berlijnse', 'ngo', 'die', 'strijd', 'voert', 'tegen', 'corruptie', 'en', 'jaarlijks', 'een', 'Index', 'van', 'Corruptie', 'Perceptie', 'opstelt', ',', 'wil', 'dat', 'de', 'Wereldbank', 'een', 'tienpuntenplan', 'uitvoert', 'om', 'ontwikkelingslanden', 'die', 'geld', 'bij', 'haar', 'lenen', 'op', 'het', 'rechte', 'pad', 'te', 'houden', '.']
Real tags: ['B-ORG', 'I-ORG', 'O', 'B-ORG', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-MISC', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-MISC', 'I-MISC', 'I-MISC', 'I-MISC', 'O', 'O', 'O', 'O', 'O', 'B-ORG', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
Predicted tags: ['B-MISC', 'I-MISC', 'O', 'B-PER', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-MISC', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-MISC', 'O', 'B-ORG', 'I-ORG', 'O', 'O', 'O', 'O', 'O', 'B-ORG', 'O', 'O', 'O', 'O',

En aquest primer exemple hi ha molts errors a l'hora de detectar el tipus d'entitat. També trobem un patró identificat en l'espanyol, entitats molt llargues que no s'acaben de detectar bé perquè se separen.

In [169]:
import random
random.seed(11)
# Escoge un error aleatorio
error = random.choice(errors2)

idx = error[0]
entity = error[1]

sentence_bad = ned_testb[idx]

sentence = [token[0] for token in sentence_bad]

print("Sentence: ", sentence)

print("Real tags:", [tag for tag in ned_testb_tags[idx]])

print("Predicted tags:", [tag[1] for tag in prediction_ned[idx]])

Sentence:  ['Scholen', 'die', 'Test-Aankoop', 'in', 'het', 'voorjaar', 'enquêteformulieren', 'toestuurde', ',', 'kregen', 'van', 'hogerhand', 'de', 'raad', 'niet', 'mee', 'te', 'werken', '.']
Real tags: ['O', 'O', 'B-ORG', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
Predicted tags: ['O', 'O', 'B-MISC', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']


I en aquest segon exemple veiem un altre cop un cas en el qual només es falla el tipus, però la ubicació de l'entitat és perfecte. És l'error general, errada a l'hora d'identificar el tipus.

### Conclusions del model holandès

El model holandès va obtenir resultats pitjors que el model espanyol, però gens dolents. En aquest cas 3 de cada 4 entitats són identificades a la perfecció, i un molt alt percentatge de les entitats s'ubiquen encara que no es trobi el tipus exacte.

Torna a ser especialment bo en detectar persones, i molt bo també en les localitzacions. Però en aquest cas el model si té més problemes per identificar les organitzacions. El tipus MISC s'identifica bastant millor que en l'espanyol, sent l'aspecte en el qual més destaca davant l'altre model.

Els errors segueixen sent quasi sempre per no identificar correctament el tipus de l'entitat, i igual que a l'espanyol, les entitats llargues són les que més problemes porten.

En definitiva, és un bon model, que troba quasi totes les entitats i que en la majoria dels casos identifica correctament el seu tipus.